This step is to impute missing data on features.

# Import modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Define functions

In [4]:
# Define function for missing clarification
def del_missing(df,col_name):
    '''Check distribution of death rate without missing data. 
    col_name = column of missing data'''
    non_missing_crit = df[col_name].notnull()
    non_missing_df = df[non_missing_crit]
    diff = round(round(non_missing_df.hospital_death.mean()*100,2) - round(df.hospital_death.mean()*100,2),2)
    print('If delete missing records in '+ col_name +': ')
    if diff <0:
        print('The percentage of death in the new data set is ' + str(diff)+'% lower than train_df.')
    else:
        print('The percentage of death in the new data set is ' + str(diff)+'% higher than train_df: ' + str(round(non_missing_df.hospital_death.mean()*100,2))+'%.')
    

In [5]:
# Define feature function
def plot_feature(df, col):
    '''
    make plot for each features
    left, the distribution of samples on the feature
    right, the dependance of death on the feature'''
    plt.figure(figsize = (14,6))
    plt.subplot(1,2,1)
    if df[col].dtype == 'float64' or df[col].dtype == 'int64':
        df[col].value_counts().sort_index().plot()
        plt.xticks(rotation=45)
    else:
        mean=df.groupby(col)['hospital_death'].mean()
        df[col]=df[col].astype('category') 
        levels=mean.sort_values().index.tolist()
        df[col].cat.reorder_categories(levels,inplace=True)
        df[col].value_counts().plot()
        #plt.xticks(levels,rotation=45)
    plt.xlabel(col)
    plt.ylabel('Counts')
    plt.title('Distribution of '+col)
    
    plt.subplot(1,2,2)   
    if df[col].dtype == 'int64' or df[col].dtype == 'float64':
        mean_x=df.groupby(col)['hospital_death'].mean()
        mean_x.plot()
        plt.xticks(rotation=45)
    else:
        mean_x=df.groupby(col)['hospital_death'].mean().sort_values()
        levels=mean_x.sort_values().index.tolist()
        plt.scatter(mean_x.sort_values().index,mean_x)
        plt.xticks(levels,rotation=45) 
    plt.xlabel(col)
    plt.ylabel('Percent of death')
    plt.title('Percent of death by '+col)
    plt.show()

# Import data set

In [2]:
train_df=pd.read_csv('train_df.csv',index_col=0)

In [ ]:
## identify categorical variables
for col in train_df.columns.to_list():
    if train_df[col].dtype == 'object':
        train_df[col]=train_df[col].astype('category')
train_df.to_csv('train_df.csv')

In [6]:
# generate new df of selected vars:
int_cols_selected = ['elective_surgery','apache_post_operative']
numeric_cols_selected = ['age', 'gcs_eyes_apache', 'gcs_motor_apache', 'gcs_verbal_apache', 'd1_diasbp_min', 'd1_diasbp_noninvasive_min', 'd1_mbp_min', 'd1_mbp_noninvasive_min', 'd1_sysbp_min', 'd1_sysbp_noninvasive_min', 'h1_diasbp_noninvasive_min', 'h1_mbp_min', 'h1_mbp_noninvasive_min', 'h1_sysbp_min', 'h1_sysbp_noninvasive_min', 'd1_calcium_min','d1_calcium_max', 'albumin_apache', 'd1_albumin_max', 'd1_albumin_min']
categorical_cols_selected = ['ethnicity','gender','icu_admit_source','hospital_admit_source','icu_stay_type','icu_type','apache_3j_bodysystem','apache_2_bodysystem']

var_list = ['hospital_death'] + int_cols_selected + numeric_cols_selected + categorical_cols_selected

train_df_miss = train_df[var_list]


In [7]:
train_df_miss.head()

,hospital_death,elective_surgery,apache_post_operative,age,gcs_eyes_apache,gcs_motor_apache,gcs_verbal_apache,d1_diasbp_min,d1_diasbp_noninvasive_min,d1_mbp_min,...,d1_albumin_max,d1_albumin_min,ethnicity,gender,icu_admit_source,hospital_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem
0,0,0,0,68.0,3.0,6.0,4.0,37.0,37.0,46.0,...,2.3,2.3,Caucasian,M,Floor,Floor,admit,CTICU,Sepsis,Cardiovascular
1,0,0,0,77.0,1.0,3.0,1.0,31.0,31.0,38.0,...,1.6,1.6,Caucasian,F,Floor,Floor,admit,Med-Surg ICU,Respiratory,Respiratory
2,0,0,0,25.0,3.0,6.0,5.0,48.0,48.0,68.0,...,NaN,NaN,Caucasian,F,Accident & Emergency,Emergency Department,admit,Med-Surg ICU,Metabolic,Metabolic
3,0,1,1,81.0,4.0,6.0,5.0,42.0,42.0,84.0,...,NaN,NaN,Caucasian,F,Operating Room / Recovery,Operating Room,admit,CTICU,Cardiovascular,Cardiovascular
4,0,0,0,19.0,NaN,NaN,NaN,57.0,57.0,90.0,...,NaN,NaN,Caucasian,M,Accident & Emergency,NaN,admit,Med-Surg ICU,Trauma,Trauma


# Missing data

In [8]:
# % of missing
pd.set_option('display.max_rows',None) 
per_missing = train_df_miss.apply(lambda col:sum(col.isnull())/col.size*100)
to_impute=per_missing[per_missing!=0].index.to_list()
print('Vars of >0 percent of missing values:')
print(per_missing[per_missing!=0])

vars of >0 percent of missing values:
age                           4.610033
gcs_eyes_apache               2.072770
gcs_motor_apache              2.072770
gcs_verbal_apache             2.072770
d1_diasbp_min                 0.179909
d1_diasbp_noninvasive_min     1.133972
d1_mbp_min                    0.239879
d1_mbp_noninvasive_min        1.612639
d1_sysbp_min                  0.173367
d1_sysbp_noninvasive_min      1.119798
h1_diasbp_noninvasive_min     8.014131
h1_mbp_min                    5.058171
h1_mbp_noninvasive_min        9.904812
h1_sysbp_min                  3.937283
h1_sysbp_noninvasive_min      8.004318
d1_calcium_min               14.249888
d1_calcium_max               14.249888
albumin_apache               59.292576
d1_albumin_max               53.532215
d1_albumin_min               53.532215
ethnicity                     1.521049
gender                        0.027259
icu_admit_source              0.122120
hospital_admit_source        23.343474
apache_3j_bodysystem      

In [9]:
# Whether deleting var will affect hospital death rate?
for col in to_impute:
    del_missing(train_df_miss,col)

If delete missing records in age: 
The percentage of death in the new data set is -0.32% lower than train_df.
If delete missing records in gcs_eyes_apache: 
The percentage of death in the new data set is -0.18% lower than train_df.
If delete missing records in gcs_motor_apache: 
The percentage of death in the new data set is -0.18% lower than train_df.
If delete missing records in gcs_verbal_apache: 
The percentage of death in the new data set is -0.18% lower than train_df.
If delete missing records in d1_diasbp_min: 
The percentage of death in the new data set is -0.02% lower than train_df.
If delete missing records in d1_diasbp_noninvasive_min: 
The percentage of death in the new data set is -0.02% lower than train_df.
If delete missing records in d1_mbp_min: 
The percentage of death in the new data set is -0.01% lower than train_df.
If delete missing records in d1_mbp_noninvasive_min: 
The percentage of death in the new data set is 0.0% higher than train_df: 8.63%.
If delete missing

So albumin_apache, d1_albumin_max,d1_albumin_min are affecting hospital_death more than other vars. 

# Imputation

In [12]:
train_df_impute=train_df_miss

In [13]:
for var in to_impute:
    if train_df_impute[var].dtypes == 'float64':
        train_df_impute[var].fillna(train_df_impute[var].mean(),inplace=True)

train_df_impute.info(verbose=True, null_counts=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 91713 entries, 0 to 91712
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   hospital_death             91713 non-null  int64   
 1   elective_surgery           91713 non-null  int64   
 2   apache_post_operative      91713 non-null  int64   
 3   age                        91713 non-null  float64 
 4   gcs_eyes_apache            91713 non-null  float64 
 5   gcs_motor_apache           91713 non-null  float64 
 6   gcs_verbal_apache          91713 non-null  float64 
 7   d1_diasbp_min              91713 non-null  float64 
 8   d1_diasbp_noninvasive_min  91713 non-null  float64 
 9   d1_mbp_min                 91713 non-null  float64 
 10  d1_mbp_noninvasive_min     91713 non-null  float64 
 11  d1_sysbp_min               91713 non-null  float64 
 12  d1_sysbp_noninvasive_min   91713 non-null  float64 
 13  h1_diasbp_noninvasive_min  9171

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [1]:
# feature before & after imputation
plot_feature(train_df_imputed,'albumin_apache')
plot_feature(train_df,'albumin_apache')


In [2]:
# feature before & after imputation
plot_feature(train_df_imputed,'d1_albumin_max')
plot_feature(train_df,'d1_albumin_max')


In [3]:
# feature before & after imputation
plot_feature(train_df_imputed,'d1_albumin_min')
plot_feature(train_df,'d1_albumin_min')


In [15]:
train_df_impute.to_csv('train_df_imputed.csv')